# Data Examples
This is an example of the data in various categories in the statsapi module

https://github.com/toddrob99/MLB-StatsAPI/wiki

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, gc, io, math, random, datetime

import statsapi


In [88]:
player = statsapi.lookup_player('nola,')
player

[{'id': 605400,
  'fullName': 'Aaron Nola',
  'firstName': 'Aaron',
  'lastName': 'Nola',
  'primaryNumber': '27',
  'currentTeam': {'id': 143},
  'primaryPosition': {'code': '1', 'abbreviation': 'P'},
  'useName': 'Aaron',
  'boxscoreName': 'Nola, Aa',
  'nickName': 'Nols',
  'mlbDebutDate': '2015-07-21',
  'nameFirstLast': 'Aaron Nola',
  'firstLastName': 'Aaron Nola',
  'lastFirstName': 'Nola, Aaron',
  'lastInitName': 'Nola, A',
  'initLastName': 'A Nola',
  'fullFMLName': 'Aaron Michael Nola',
  'fullLFMName': 'Nola, Aaron Michael'}]

## Team Data
This is data available on teams. 

In [55]:
#Team Metadata
teams = statsapi.get('teams',{'sportIds':1,'activeStatus':'Yes'})['teams']
teams = pd.DataFrame(teams).drop(columns=['springLeague','springVenue','sport'])
teams['league'] = teams['league'].apply(lambda x: x['name'])
teams['division'] = teams['division'].apply(lambda x: x['name'])
teams['venue'] = teams['venue'].apply(lambda x: x['name'])
display(teams.head(5))

#roster
roster = statsapi.roster(110)
roster_df = roster.split('\n')
roster_df = pd.DataFrame([x.split('  ') for x in roster_df], columns=['number','position','name']).dropna()
roster_df['id'] = roster_df['name'].apply(lambda x: statsapi.lookup_player(x.split(' ')[1])[0]['id'])
display(roster_df.head(5))

,allStarStatus,id,name,link,season,venue,teamCode,fileCode,abbreviation,teamName,locationName,firstYearOfPlay,league,division,shortName,franchiseName,clubName,active
0,N,133,Oakland Athletics,/api/v1/teams/133,2024,Oakland Coliseum,oak,oak,OAK,Athletics,Oakland,1901,American League,American League West,Oakland,Oakland,Athletics,True
1,N,134,Pittsburgh Pirates,/api/v1/teams/134,2024,PNC Park,pit,pit,PIT,Pirates,Pittsburgh,1882,National League,National League Central,Pittsburgh,Pittsburgh,Pirates,True
2,N,135,San Diego Padres,/api/v1/teams/135,2024,Petco Park,sdn,sd,SD,Padres,San Diego,1968,National League,National League West,San Diego,San Diego,Padres,True
3,N,136,Seattle Mariners,/api/v1/teams/136,2024,T-Mobile Park,sea,sea,SEA,Mariners,Seattle,1977,American League,American League West,Seattle,Seattle,Mariners,True
4,N,137,San Francisco Giants,/api/v1/teams/137,2024,Oracle Park,sfn,sf,SF,Giants,San Francisco,1883,National League,National League West,San Francisco,San Francisco,Giants,True


,number,position,name,id
0,#35,C,Adley Rutschman,668939
1,#25,RF,Anthony Santander,623993
2,#21,LF,Austin Hays,669720
3,#31,CF,Cedric Mullins,656775
4,#19,P,Cole Irvin,671096


## Hitter Data

Data available on hitters throught player_stat_data

In [145]:
o_team = pd.DataFrame()
o_hitting = pd.DataFrame()
stat = pd.DataFrame()

for player in roster_df['id']:
    #options for type include career, season, yearByYear
    player_data = statsapi.player_stat_data(player,group='hitting',type='career')
    meta = pd.DataFrame(player_data).drop(columns='stats')
    if meta.loc[:,'current_team'].values == 'Baltimore Orioles':
        o_team = pd.concat([o_team,meta])
        try:
            s = pd.DataFrame([player_data['stats'][0]['stats']])
            s['id'] = player
            stat = pd.concat([stat,s],axis=0)
        except:
            print(player_data)

    

display(o_team.head(5))
o_hitting = pd.merge(o_team,stat,how='left',on='id')
display(o_hitting.head(5))
o_team.set_index('id',inplace=True)

C:\Users\batuu\AppData\Local\Temp\ipykernel_10444\4181441386.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if meta.loc[:,'current_team'].values == 'Baltimore Orioles':
C:\Users\batuu\AppData\Local\Temp\ipykernel_10444\4181441386.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if meta.loc[:,'current_team'].values == 'Baltimore Orioles':
C:\Users\batuu\AppData\Local\Temp\ipykernel_10444\4181441386.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if meta.loc[:,'current_team'].values == 'Baltimore Orioles':


,id,first_name,last_name,active,current_team,position,nickname,last_played,mlb_debut,bat_side,pitch_hand
0,668939,Adley,Rutschman,True,Baltimore Orioles,C,None,None,2022-05-21,Switch,Right
0,623993,Anthony,Santander,True,Baltimore Orioles,RF,Agua Blanca,None,2017-08-18,Switch,Right
0,669720,Austin,Hays,True,Baltimore Orioles,LF,None,None,2017-09-07,Right,Right
0,656775,Cedric,Mullins,True,Baltimore Orioles,CF,None,None,2018-08-10,Left,Left
0,681297,Colton,Cowser,True,Baltimore Orioles,LF,None,None,2023-07-05,Left,Right


,id,first_name,last_name,active,current_team,position,nickname,last_played,mlb_debut,bat_side,...,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun
0,668939,Adley,Rutschman,True,Baltimore Orioles,C,None,None,2022-05-21,Switch,...,1187,444,126,375,0,8,.301,0.90,0,30.67
1,623993,Anthony,Santander,True,Baltimore Orioles,RF,Agua Blanca,None,2017-08-18,Switch,...,2480,1041,342,1002,2,18,.273,0.60,0,20.04
2,669720,Austin,Hays,True,Baltimore Orioles,LF,None,None,2017-09-07,Right,...,1972,785,229,728,3,12,.305,0.94,0,28.78
3,656775,Cedric,Mullins,True,Baltimore Orioles,CF,None,None,2018-08-10,Left,...,2243,864,229,692,13,14,.295,0.78,3,28.71
4,681297,Colton,Cowser,True,Baltimore Orioles,LF,None,None,2023-07-05,Left,...,87,16,5,28,0,1,.255,1.19,0,-.--


## Fielding Data

Data on fielding 

In [157]:
o_fielding = pd.DataFrame()
stat = pd.DataFrame()

for player in o_team.index:
    player_data = statsapi.player_stat_data(player,group='fielding',type='career')
    position = o_team.loc[player,'position']
    try:
        for p in player_data['stats']:
            if p['stats']['position']['abbreviation'] == position:
                s = pd.DataFrame([p['stats']])
                s['id'] = player
                stat = pd.concat([stat,s],axis=0)
    except Exception as e:
        print(e)

o_fielding = pd.merge(o_team,stat,how='left',on='id')
display(o_fielding.head(5))


,first_name,last_name,active,current_team,position,nickname,last_played,mlb_debut,bat_side,pitch_hand
id,,,,,,,,,,
668939,Adley,Rutschman,True,Baltimore Orioles,C,None,None,2022-05-21,Switch,Right
623993,Anthony,Santander,True,Baltimore Orioles,RF,Agua Blanca,None,2017-08-18,Switch,Right
669720,Austin,Hays,True,Baltimore Orioles,LF,None,None,2017-09-07,Right,Right
656775,Cedric,Mullins,True,Baltimore Orioles,CF,None,None,2018-08-10,Left,Left
681297,Colton,Cowser,True,Baltimore Orioles,LF,None,None,2023-07-05,Left,Right


,id,first_name,last_name,active,current_team,position_x,nickname,last_played,mlb_debut,bat_side,...,innings,games,passedBall,doublePlays,triplePlays,catcherERA,catchersInterference,wildPitches,throwingErrors,pickoffs
0,668939,Adley,Rutschman,True,Baltimore Orioles,C,None,None,2022-05-21,Switch,...,1739.2,208,2.0,8,0,3.87,4.0,63.0,10,1.0
1,623993,Anthony,Santander,True,Baltimore Orioles,RF,Agua Blanca,None,2017-08-18,Switch,...,3143.2,390,NaN,4,0,NaN,NaN,NaN,5,NaN
2,669720,Austin,Hays,True,Baltimore Orioles,LF,None,None,2017-09-07,Right,...,2646.2,327,NaN,0,0,NaN,NaN,NaN,0,NaN
3,656775,Cedric,Mullins,True,Baltimore Orioles,CF,None,None,2018-08-10,Left,...,4329.0,527,NaN,9,0,NaN,NaN,NaN,4,NaN
4,681297,Colton,Cowser,True,Baltimore Orioles,LF,None,None,2023-07-05,Left,...,64.0,9,NaN,0,0,NaN,NaN,NaN,0,NaN


## Pitching Data

Data on pitchers

In [160]:
o_pitching = pd.DataFrame()
stat = pd.DataFrame()

for player in o_team[o_team['position']=='P'].index:
    player_data = statsapi.player_stat_data(player,group='pitching',type='career')
    try:
        s = pd.DataFrame([player_data['stats'][0]['stats']])
        s['id'] = player
        stat = pd.concat([stat,s],axis=0)
    except Exception as e:
            print(e)

    

o_pitching = pd.merge(o_team[o_team['position']=='P'],stat,how='left',on='id')
display(o_pitching)

,id,first_name,last_name,active,current_team,position,nickname,last_played,mlb_debut,bat_side,...,strikeoutsPer9Inn,walksPer9Inn,hitsPer9Inn,runsScoredPer9,homeRunsPer9,inheritedRunners,inheritedRunnersScored,catchersInterference,sacBunts,sacFlies
0,669203,Corbin,Burnes,True,Baltimore Orioles,P,Burnesy,None,2018-07-10,Right,...,11.03,2.57,6.89,3.47,0.95,25,5,0,5,13
1,518886,Craig,Kimbrel,True,Baltimore Orioles,P,Dirty Craig,None,2010-05-07,Right,...,14.18,3.65,5.25,2.67,0.78,97,18,1,8,12
2,543056,Danny,Coulombe,True,Baltimore Orioles,P,Frenchie,None,2014-09-16,Left,...,8.78,3.35,7.98,3.97,0.91,150,41,0,4,6
3,622253,Dillon,Tate,True,Baltimore Orioles,P,None,None,2019-07-29,Right,...,7.22,2.77,7.27,4.35,0.84,93,32,0,2,11
4,669211,Keegan,Akin,True,Baltimore Orioles,P,None,None,2020-08-14,Left,...,8.83,3.04,9.54,5.68,1.26,43,17,1,2,10
5,657508,Mike,Baumann,True,Baltimore Orioles,P,None,None,2021-09-07,Right,...,7.39,4.02,8.92,4.98,1.04,54,21,0,1,3
6,666974,Yennier,Cano,True,Baltimore Orioles,P,None,None,2022-05-11,Right,...,8.62,2.84,8.43,3.98,0.66,39,8,0,1,2
